In [8]:
import tweepy
from tweepy import OAuthHandler,Stream,API
from tweepy.streaming import StreamListener
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import re
import twitter_creds
from twitter_creds import *
import re
import json
from collections import Counter
import pandas as pd
from langdetect import detect
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
from wordcloud import WordCloud
import numpy as np
from nltk.corpus import stopwords
from string import punctuation

In [2]:

def Authentication() : 
    auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth)
    return api
    

In [5]:
class MyStreamListener(StreamListener):
    def on_data(self,data):
        try:
            with open('avengers_endgame.json','a') as ff:
                ff.write(data)
            return True
        except BaseException as e:
            print("Error on data : %s" % str(e))
            return True
    def on_error(self,status):
        print(status)
        return True         
    

In [6]:
api = Authentication()
ListenObj = MyStreamListener()
my_stream = Stream(api.auth,ListenObj)

In [7]:
my_stream.filter(track=['𝙰𝚟𝚎𝚗𝚐𝚎𝚛𝚜𝙴𝚗𝚍𝚐𝚊𝚖𝚎','avengersendgame','endgame'],languages = ["en"],async=True)

In [9]:
emoticon_str = r"""
    (?: 
    [:=;] ##eyes
    [oO\-]? ##Nose
    [O\)\]
    )
"""

In [10]:
emoticon_str

'\n    (?: \n    [:=;] ##eyes\n    [oO\\-]? ##Nose\n    [O\\)\\]\n    )\n'

In [44]:
regex_str = [
r'<[^>]+>', ##HTML Tags
r'(?:@[\w_]+)', ##@-mentions 
r"(?:\#+[\w_]+[\w\'_\-]”[\w_]+)", #hash tags
r'http[s]?://(?:[a-z][0-9]|[$-_@.&+]|[|*\(\),]|(?:%[0-9a-f][0-9a-f]))+', ##urls
r'(?:(?:\d+,?)+(?:\.?\d+)?)', ##number
r"(?:[a-z][a-z'\-_]+[a-z])", ##words with - and ‘
r'(?:[\w_]+)', ##Other words
r'(?:\S)' #Anything else
   ]


In [35]:
regex_str = [r'<[^>]+>']

In [45]:
token_re = re.compile('('+'|'.join(regex_str)+')',re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r"^'+emoticon_str+'$", re.VERBOSE | re.IGNORECASE)

In [46]:
token_re

re.compile(r"(<[^>]+>|(?:@[\w_]+)|(?:\#+[\w_]+[\w\'_\-]”[\w_]+)|http[s]?://(?:[a-z][0-9]|[$-_@.&+]|[|*\(\),]|(?:%[0-9a-f][0-9a-f]))+|(?:(?:\d+,?)+(?:\.?\d+)?)|(?:[a-z][a-z'\-_]+[a-z])|(?:[\w_]+)|(?:\S))",
re.IGNORECASE|re.UNICODE|re.VERBOSE)

In [14]:
def tokenize(s):
    return token_re.findall(s)


In [25]:
def preprocess(s):
    tokens = tokenize(s)
    tokens = [token if token in emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [16]:
punctuation = list(punctuation)

In [17]:
stop = stopwords.words('english') + punctuation + ['rt','RT','via']


In [18]:
fname = '/Users/rahuljain/Desktop/Python/MachineLearningwithPython/Sentiment_Analysis/avengers_endgame.json'

In [21]:
with open(fname,'r',newline='\r\n') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        ##print(line)
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        terms_set  = set(terms_stop)
        ##terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
        terms_only = [term for term in preprocess(tweet['text'].lower()) if term not in stop]
        terms_all  = [term for term in preprocess(tweet['text'])]
        new_terms = []
        for term in terms_only:
            if len(term) > 3:
                new_terms.append(term)
                

{"created_at":"Sun Apr 07 16:00:37 +0000 2019","id":1114920926024540162,"id_str":"1114920926024540162","text":"@y_sulaimon @AbkTrauma Advengers endgame","display_text_range":[23,40],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":1114853183858384896,"in_reply_to_status_id_str":"1114853183858384896","in_reply_to_user_id":1051161770147106816,"in_reply_to_user_id_str":"1051161770147106816","in_reply_to_screen_name":"y_sulaimon","user":{"id":1099658884615233536,"id_str":"1099658884615233536","name":"im_ola__dimple_guy\ud83d\ude0e","screen_name":"Agboola17861422","location":"Ikorudu, Nigeria","url":"https:\/\/api.whatsapp.com\/send?phone=2348126922080&text","description":"caring,jovial,friendly..dimple_boy ,baby_face\ud83e\udd29..kind people are my kind of person \u2764\ufe0f","translator_type":"none","protected":false,"verified":false,"followers_count":260,"friends_count":46

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [20]:
new_terms

[]

In [62]:
print(count_all.most_common(50))

[]


In [64]:
df = pd.DataFrame()

In [65]:
df.columns = ('terms','freq')

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [22]:
term = '"created_at":"Sun Apr 07 16:00:37 +0000 2019","id":1114920926024540162,"id_str":"1114920926024540162","text":"@y_sulaimon @AbkTrauma Advengers endgame","display_text_range":[23,40],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":1114853183858384896,"in_reply_to_status_id_str":"1114853183858384896","in_reply_to_user_id":1051161770147106816,"in_reply_to_user_id_str":"1051161770147106816","in_reply_to_screen_name":"y_sulaimon","user":{"id":1099658884615233536,"id_str":"1099658884615233536","name":"im_ola__dimple_guy\ud83d\ude0e","screen_name":"Agboola17861422","location":"Ikorudu, Nigeria","url":"https:\/\/api.whatsapp.com\/send?phone=2348126922080&text","description":"caring,jovial,friendly..dimple_boy ,baby_face\ud83e\udd29..kind people are my kind of person \u2764\ufe0f","translator_type":"none","protected":false,"verified":false,"followers_count":260,"friends_count":463,"listed_count":0,"favourites_count":554,"statuses_count":110,"created_at":"Sun Feb 24 13:14:43 +0000 2019","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"1DA1F2","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images'

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 612-613: surrogates not allowed

In [27]:
print(preprocess(text))

[]


In [28]:
text

'{"created_at":"Sun Apr 07 16:00:37 +0000 2019","id":1114920926024540162,"id_str":"1114920926024540162","text":"@y_sulaimon @AbkTrauma Advengers endgame","display_text_range":[23,40],"source":"<a href="http:\\/\\/twitter.com\\/download\\/iphone" rel="nofollow">Twitter for iPhone<\\/a>","truncated":false,"in_reply_to_status_id":1114853183858384896,"in_reply_to_status_id_str":"1114853183858384896","in_reply_to_user_id":1051161770147106816,"in_reply_to_user_id_str":"1051161770147106816","in_reply_to_screen_name":"y_sulaimon","user":{"id":1099658884615233536,"id_str":"1099658884615233536","name":"im_ola__dimple_guy\ud83d\ude0e","screen_name":"Agboola17861422","location":"Ikorudu, Nigeria","url":"https:\\/\\/api.whatsapp.com\\/send?phone=2348126922080&text","description":"caring,jovial,friendly..dimple_boy ,baby_face\ud83e\udd29..kind people are my kind of person ❤️","translator_type":"none","protected":false,"verified":false,"followers_count":260,"friends_count":463,"listed_count":0,"favou

In [47]:
tokens = tokenize(text)

In [48]:
tokens

['{',
 '"',
 'created_at',
 '"',
 ':',
 '"',
 'Sun',
 'Apr',
 '07',
 '16',
 ':',
 '00',
 ':',
 '37',
 '+',
 '0000',
 '2019',
 '"',
 ',',
 '"',
 'id',
 '"',
 ':',
 '1114920926024540162,',
 '"',
 'id_str',
 '"',
 ':',
 '"',
 '1114920926024540162',
 '"',
 ',',
 '"',
 'text',
 '"',
 ':',
 '"',
 '@y_sulaimon',
 '@AbkTrauma',
 'Advengers',
 'endgame',
 '"',
 ',',
 '"',
 'display_text_range',
 '"',
 ':',
 '[',
 '23,40',
 ']',
 ',',
 '"',
 'source',
 '"',
 ':',
 '"',
 '<a href="http:\\/\\/twitter.com\\/download\\/iphone" rel="nofollow">',
 'Twitter',
 'for',
 'iPhone',
 '<\\/a>',
 '"',
 ',',
 '"',
 'truncated',
 '"',
 ':',
 'false',
 ',',
 '"',
 'in_reply_to_status_id',
 '"',
 ':',
 '1114853183858384896,',
 '"',
 'in_reply_to_status_id_str',
 '"',
 ':',
 '"',
 '1114853183858384896',
 '"',
 ',',
 '"',
 'in_reply_to_user_id',
 '"',
 ':',
 '1051161770147106816,',
 '"',
 'in_reply_to_user_id_str',
 '"',
 ':',
 '"',
 '1051161770147106816',
 '"',
 ',',
 '"',
 'in_reply_to_screen_name',
 '"',
 ':',
 